In [1]:
%matplotlib inline
import pandas as pd
from sklearn import tree
from sklearn.cross_validation import cross_val_score

# Load in data and create sets. dropping all na columns on the live data set.
lemons = pd.read_csv('../DAT18NYC/data/lemons.csv')
lemons_oos = pd.read_csv('../DAT18NYC/data/lemons_oos.csv')
print lemons.dtypes

RefId                                  int64
IsBadBuy                               int64
PurchDate                             object
Auction                               object
VehYear                                int64
VehicleAge                             int64
Make                                  object
Model                                 object
Trim                                  object
SubModel                              object
Color                                 object
Transmission                          object
WheelTypeID                          float64
WheelType                             object
VehOdo                                 int64
Nationality                           object
Size                                  object
TopThreeAmericanName                  object
MMRAcquisitionAuctionAveragePrice    float64
MMRAcquisitionAuctionCleanPrice      float64
MMRAcquisitionRetailAveragePrice     float64
MMRAcquisitonRetailCleanPrice        float64
MMRCurrent

In [136]:
lemons = lemons.drop('AUCGUART', 1)
lemons_oos = lemons_oos.drop('AUCGUART', 1)
lemons = lemons.drop('PRIMEUNIT', 1)
lemons_oos = lemons_oos.drop('PRIMEUNIT', 1)
lemons = lemons.dropna(axis=0)
lemons_oos = lemons_oos.dropna(axis=0)
# Generating a list of continuous data features from the describe dataframe. 
# Then, removing the two non-features (RefId is an index, IsBadBuy is the prediction value)
features = list(lemons.describe().columns)
features.remove('RefId')
#features.remove('IsBadBuy')
print features

['IsBadBuy', 'VehYear', 'VehicleAge', 'WheelTypeID', 'VehOdo', 'MMRAcquisitionAuctionAveragePrice', 'MMRAcquisitionAuctionCleanPrice', 'MMRAcquisitionRetailAveragePrice', 'MMRAcquisitonRetailCleanPrice', 'MMRCurrentAuctionAveragePrice', 'MMRCurrentAuctionCleanPrice', 'MMRCurrentRetailAveragePrice', 'MMRCurrentRetailCleanPrice', 'BYRNO', 'VNZIP1', 'VehBCost', 'IsOnlineSale', 'WarrantyCost']


In [137]:
# feature selection
from sklearn import feature_selection as f_select

y = lemons.IsBadBuy

significant_features = []
pvals = []
for feature in features:
    pval = f_select.f_classif(lemons[feature], y)
    if pval[1][0] < 0.05:
        significant_features.append(feature)
        pvals.append(pval[1][0])

In [138]:
print significant_features

new_lemons = lemons[significant_features]
new_lemons.head()

['IsBadBuy', 'VehYear', 'VehicleAge', 'WheelTypeID', 'VehOdo', 'MMRAcquisitionAuctionAveragePrice', 'MMRAcquisitionAuctionCleanPrice', 'MMRAcquisitionRetailAveragePrice', 'MMRAcquisitonRetailCleanPrice', 'MMRCurrentAuctionAveragePrice', 'MMRCurrentAuctionCleanPrice', 'MMRCurrentRetailAveragePrice', 'MMRCurrentRetailCleanPrice', 'BYRNO', 'VehBCost', 'IsOnlineSale', 'WarrantyCost']


,IsBadBuy,VehYear,VehicleAge,WheelTypeID,VehOdo,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,BYRNO,VehBCost,IsOnlineSale,WarrantyCost
0,0,2006,3,1,89046,8155,9829,11636,13600,7451,8552,11597,12409,21973,7100,0,1113
1,0,2004,5,1,93593,6854,8383,10897,12572,7456,9222,11374,12791,19638,7600,0,1053
2,0,2005,4,2,69367,3913,5054,7723,8707,3247,4384,6739,7911,19638,4000,0,1020
3,0,2004,5,2,81054,3901,4908,6706,8577,4709,5827,8149,9451,19638,5600,0,594
4,0,2004,5,2,65328,2966,4038,6240,8496,2980,4115,6230,8603,19638,4200,0,533


In [139]:
new_lemons.describe()

,IsBadBuy,VehYear,VehicleAge,WheelTypeID,VehOdo,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,BYRNO,VehBCost,IsOnlineSale,WarrantyCost
count,47036.000000,47036.000000,47036.000000,47036.000000,47036.000000,47036.000000,47036.000000,47036.000000,47036.000000,47036.000000,47036.000000,47036.000000,47036.000000,47036.000000,47036.000000,47036.000000,47036.000000
mean,0.094736,2005.356876,4.163024,1.493686,71779.831108,6167.942512,7419.721766,8543.512969,9903.858002,6174.286419,7438.419785,8823.359129,10196.336508,26503.365337,6753.628804,0.024683,1279.590356
std,0.292853,1.724427,1.704189,0.521773,14412.869852,2460.879467,2722.657916,3160.889958,3391.129086,2431.270552,2681.131657,3088.632758,3306.870091,25966.906756,1752.077351,0.155160,582.290447
min,0.000000,2001.000000,0.000000,1.000000,5368.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,835.000000,1400.000000,0.000000,462.000000
25%,0.000000,2004.000000,3.000000,1.000000,62180.500000,4312.000000,5459.750000,6311.000000,7519.000000,4316.000000,5471.000000,6565.000000,7822.000000,17212.000000,5465.000000,0.000000,853.000000
50%,0.000000,2005.000000,4.000000,1.000000,73603.500000,6166.000000,7383.000000,8494.000000,9857.500000,6130.000000,7387.000000,8811.000000,10175.000000,19662.000000,6745.000000,0.000000,1169.000000
75%,0.000000,2007.000000,5.000000,2.000000,82545.000000,7810.000000,9056.000000,10710.000000,12158.000000,7781.000000,9055.000000,10976.000000,12376.000000,22808.000000,7915.000000,0.000000,1623.000000
max,1.000000,2010.000000,9.000000,3.000000,115717.000000,35722.000000,36859.000000,39080.000000,40308.000000,35722.000000,36859.000000,39080.000000,40308.000000,99761.000000,35900.000000,1.000000,7498.000000


In [140]:
new_lemons['AuctionAvgDiff'] = new_lemons.MMRAcquisitionAuctionAveragePrice - new_lemons.MMRCurrentAuctionAveragePrice
new_lemons['AuctionCleanDiff'] = new_lemons.MMRAcquisitionAuctionCleanPrice - new_lemons.MMRCurrentAuctionCleanPrice
new_lemons['RetailAvgDiff'] = new_lemons.MMRAcquisitionRetailAveragePrice - new_lemons.MMRCurrentRetailAveragePrice

lemons_oos['AuctionAvgDiff'] = lemons_oos.MMRAcquisitionAuctionAveragePrice - lemons_oos.MMRCurrentAuctionAveragePrice
lemons_oos['AuctionCleanDiff'] = lemons_oos.MMRAcquisitionAuctionCleanPrice - lemons_oos.MMRCurrentAuctionCleanPrice
lemons_oos['RetailAvgDiff'] = lemons_oos.MMRAcquisitionRetailAveragePrice - lemons_oos.MMRCurrentRetailAveragePrice
#new_lemons['RetailCleanDiff'] = new_lemons.MMRAcquisitionRetailCleanPrice - new_lemons.MMRCurrentRetailCleanPrice

c:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
c:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
c:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: 

In [141]:
new_lemons.drop('BYRNO', axis=1, inplace=True)
lemons_oos.drop('BYRNO', axis=1, inplace=True)

c:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [142]:
clean_lemons = new_lemons[['VehYear',
 'VehicleAge',
 'WheelTypeID',
 'VehOdo',  
 'VehBCost',
 'IsOnlineSale',
 'WarrantyCost',
 'AuctionAvgDiff',
 'AuctionCleanDiff',
 'RetailAvgDiff']]

lemons_oos = lemons_oos[['VehYear',
 'VehicleAge',
 'WheelTypeID',
 'VehOdo',  
 'VehBCost',
 'IsOnlineSale',
 'WarrantyCost',
 'AuctionAvgDiff',
 'AuctionCleanDiff',
 'RetailAvgDiff',
 'RefId']]

y = new_lemons['IsBadBuy']

In [143]:
best_score = -1
for depth in range(1, 10):
    scores = cross_val_score(tree.DecisionTreeClassifier(max_depth=depth, random_state=1234),
                                clean_lemons,
                                y,
                                scoring='roc_auc',
                                cv=5)
    if scores.mean() > best_score:
        best_depth = depth
        best_score = scores.mean()

print best_score
print best_depth

0.654150762814
5


In [144]:
# Create a classifier and prediction.
clf = tree.DecisionTreeClassifier(max_depth=5, random_state=1234).fit(clean_lemons, y)

y_pred = clf.predict(lemons_oos[['VehYear',
 'VehicleAge',
 'WheelTypeID',
 'VehOdo',  
 'VehBCost',
 'IsOnlineSale',
 'WarrantyCost',
 'AuctionAvgDiff',
 'AuctionCleanDiff',
 'RetailAvgDiff']])

# Create a submission
submission = pd.DataFrame({ 'RefId' : lemons_oos.RefId, 'prediction' : y_pred })
submission.to_csv('submission.csv')

IOError: [Errno 13] Permission denied: 'submission.csv'